In [1]:
import torch
import numpy as np
from sys import getsizeof


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
dtype = np.complex64

N, M = 10, 20
dt = 0.2

Using cuda device


In [8]:
a_np = np.array(np.random.rand(N, N, N, N, N, N) * 10, dtype=dtype)
a_cpu = torch.tensor(a_np, device="cpu")
a_gpu = torch.tensor(a_np, device=device)

bs_np = [np.array(np.random.rand(N, N) * 10, dtype=dtype) for _ in range(a_np.ndim)]
bs_np = [(b + b.T)*0.5 for b in bs_np]
bs_cpu = [torch.tensor(b, device="cpu") for b in bs_np]
bs_gpu = [torch.tensor(b, device=device) for b in bs_np]


- Testing the partial transforms in CPU using pytorch:

In [11]:
%%time
a = a_cpu
for i, b in enumerate(bs_cpu):
    u = torch.matrix_exp(-1.0j * dt * b)
    a = torch.tensordot(a, u, ([i], [0]))
    a = torch.moveaxis(a, -1, i)
print(a.shape, a.device)

torch.Size([10, 10, 10, 10, 10, 10]) cpu
CPU times: user 95.4 ms, sys: 9.01 ms, total: 104 ms
Wall time: 9.45 ms


- Testing the partial transforms in GPU using pytorch:

In [10]:
%%time
a = a_gpu
for i, b in enumerate(bs_gpu):
    u = torch.matrix_exp(-1.0j * dt * b)
    a = torch.tensordot(a, u, ([i], [0]))
    a = torch.moveaxis(a, -1, i)
print(a.shape, a.device)

torch.Size([10, 10, 10, 10, 10, 10]) cuda:0
CPU times: user 21 ms, sys: 50 µs, total: 21 ms
Wall time: 18.3 ms
